In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

Set the Pandas defaults

In [2]:
pd.set_option('mode.chained_assignment', None)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

## Read Data

In [3]:
zillow = pd.read_parquet('data/zillow.parquet')
sld = pd.read_parquet('data/Smart_Location_Database.parquet')
food_atlas = pd.read_parquet('data/food_atlas.parquet')
fips = pd.read_parquet('data/fips.parquet')

In [4]:
# Fix the SLD tract data
sld['StCtyTract'] = sld['GEOID10'].astype('str').str[:-1].str.zfill(11)

# Fix the fips dataframe's zfill
fips['FIPS'] = fips['FIPS'].astype('str').str.zfill(5)

In [5]:
# The XGB Predictions Results dataframe
xgb_results = pd.read_parquet('final_output/xgb_results.parquet')

## Load SLD and FIPS details to XGB Results DataFrame

In [6]:
# Get the 4 Indexes used to build the NatWalkInd dependent variable
sld_means = sld[['FIPS','D2A_Ranked','D2B_Ranked','D3B_Ranked','D4A_Ranked']]
sld_means = sld_means.groupby('FIPS').mean()

# Merge the 4 indexes
xgb_results = pd.merge(xgb_results, sld_means, left_index=True, right_index=True, how='left')

In [7]:
# Merge some of the SLD and FIPS details to the dataframe to make lookups easier later
xgb_results = xgb_results.reset_index().rename(columns={"index":"FIPS"})
xgb_results = pd.merge(xgb_results, fips[['FIPS','State','County']], on='FIPS', how='left')
xgb_results.State.fillna("Alaska", inplace=True)
xgb_results.County.fillna("Petersburg", inplace=True)
xgb_results = pd.merge(xgb_results, sld[['FIPS','CBSA_Name']].drop_duplicates().dropna(axis=0), on='FIPS', how='left')

## Add Home Values for Annual January observations

In [8]:
home_val_df = pd.DataFrame()
home_val_df = zillow[(zillow.Y==2012) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2012'})
home_val_2013 = zillow[(zillow.Y==2013) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2013'})
home_val_2014 = zillow[(zillow.Y==2014) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2014'})
home_val_2015 = zillow[(zillow.Y==2015) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2015'})
home_val_2016 = zillow[(zillow.Y==2016) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2016'})
home_val_2017 = zillow[(zillow.Y==2017) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2017'})
home_val_2018 = zillow[(zillow.Y==2018) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2018'})
home_val_2019 = zillow[(zillow.Y==2019) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2019'})
home_val_2020 = zillow[(zillow.Y==2020) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2020'})
home_val_2021 = zillow[(zillow.Y==2021) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2021'})
home_val_2022 = zillow[(zillow.Y==2022) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2022'})
home_val_2023 = zillow[(zillow.Y==2023) & (zillow.M==1)][['FIPS','Home Value']].rename(columns={'Home Value':'HV_2023'})
home_val_df = pd.merge(home_val_df, home_val_2013, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2014, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2015, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2016, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2017, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2018, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2019, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2020, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2021, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2022, on='FIPS', how='left')
home_val_df = pd.merge(home_val_df, home_val_2023, on='FIPS', how='left')

In [9]:
xgb_results = pd.merge(xgb_results, home_val_df, on="FIPS", how="left")

## Add Annual Home Value Differences to 2023

In [10]:
xgb_results['HV_7yr_Diff'] = xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] - xgb_results[xgb_results.HV_2017.notna()]['HV_2017']
xgb_results['HV_6yr_Diff'] = xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] - xgb_results[xgb_results.HV_2017.notna()]['HV_2018']
xgb_results['HV_5yr_Diff'] = xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] - xgb_results[xgb_results.HV_2017.notna()]['HV_2019']
xgb_results['HV_4yr_Diff'] = xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] - xgb_results[xgb_results.HV_2017.notna()]['HV_2020']
xgb_results['HV_3yr_Diff'] = xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] - xgb_results[xgb_results.HV_2017.notna()]['HV_2021']

xgb_results['HV_7yr_Diff_Pct'] = (xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] / xgb_results[xgb_results.HV_2017.notna()]['HV_2017']) * 100
xgb_results['HV_6yr_Diff_Pct'] = (xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] / xgb_results[xgb_results.HV_2017.notna()]['HV_2018']) * 100
xgb_results['HV_5yr_Diff_Pct'] = (xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] / xgb_results[xgb_results.HV_2017.notna()]['HV_2019']) * 100
xgb_results['HV_4yr_Diff_Pct'] = (xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] / xgb_results[xgb_results.HV_2017.notna()]['HV_2020']) * 100
xgb_results['HV_3yr_Diff_Pct'] = (xgb_results[xgb_results.HV_2017.notna()]['HV_2023'] / xgb_results[xgb_results.HV_2017.notna()]['HV_2021']) * 100

## Add Walk Difference Values

In [11]:
# Add a column for the difference between the actuals and our predictions
xgb_results['Walk_Difference'] = (xgb_results['NatWalkInd']) - (xgb_results['Prediction'])

# Create a MinMax value for the Walk_Difference column
scaler = MinMaxScaler()
xgb_results['Scaled_Walk_Difference'] = scaler.fit_transform(xgb_results[['Walk_Difference']])
xgb_results['Walk_Difference_Pct'] = (xgb_results['Walk_Difference'] / xgb_results['NatWalkInd']) * 100

## Write out finished DataFrame

In [12]:
xgb_results.to_parquet('final_output/xgb_results_full.parquet')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e75edf0e-32f1-42b8-8a27-9dc0078a206d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>